In [1]:
import openai
import os
from util import get_api_key
openai.api_key = get_api_key()

In [2]:
schema_files = os.listdir('../schemas')

In [3]:
all_schemas = {}

In [4]:
for file in schema_files:
    opened_file = open('../schemas/' + file, 'r')
    all_schemas[file] = opened_file.read()

In [5]:
system_prompt = """You are a data engineer looking to create a slowly-changing dimension table query"""

In [6]:
user_prompt = f"""Using cumulative table input schema {all_schemas['players.sql']}
                    and expected output schema {all_schemas['players_scd_table.sql']} 
                    generate a query to do a slowly-changing dimension 
                    transformation tracking changes on the dimensions is_active and scoring_class, 
                    use markdown and SQL for the transformation
            """

In [7]:
print(system_prompt)
print(user_prompt)

You are a data engineer looking to create a slowly-changing dimension table query
Using cumulative table input schema  CREATE TYPE season_stats AS (
                         season Integer,
                         pts REAL,
                         ast REAL,
                         reb REAL,
                         weight INTEGER
                       );
 CREATE TYPE scorer_class AS
     ENUM ('bad', 'average', 'good', 'star');


 CREATE TABLE players (
     player_name TEXT,
     height TEXT,
     college TEXT,
     country TEXT,
     draft_year TEXT,
     draft_round TEXT,
     draft_number TEXT,
     seasons season_stats[],
     scoring_class scorer_class,
     is_active BOOLEAN,
     current_season INTEGER,
     PRIMARY KEY (player_name, current_season)
 );




                    and expected output schema create table players_scd_table
(
	player_name text,
	scoring_class scorer_class,
	is_active boolean,
	start_date integer,
	end_date integer,
	is_current boolean
);

 
      

In [8]:
response = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0
)
print(response.choices[0].message.content)
answer = response.choices[0].message.content

Here is a SQL query that can be used to create a slowly-changing dimension (SCD) table from the `players` table. This query tracks changes on the `is_active` and `scoring_class` dimensions. 

```sql
WITH ranked_players AS (
    SELECT 
        player_name,
        scoring_class,
        is_active,
        current_season,
        ROW_NUMBER() OVER (PARTITION BY player_name ORDER BY current_season) AS rn
    FROM players
),
changes AS (
    SELECT 
        rp1.player_name,
        rp1.scoring_class,
        rp1.is_active,
        rp1.current_season AS start_date,
        COALESCE(rp2.current_season - 1, -1) AS end_date,
        CASE WHEN rp2.player_name IS NULL THEN TRUE ELSE FALSE END AS is_current
    FROM ranked_players rp1
    LEFT JOIN ranked_players rp2 ON rp1.player_name = rp2.player_name AND rp1.rn = rp2.rn - 1
    WHERE rp1.scoring_class != COALESCE(rp2.scoring_class, rp1.scoring_class) OR rp1.is_active != COALESCE(rp2.is_active, rp1.is_active)
)
INSERT INTO players_scd_table (p

In [9]:
if not os.path.exists('output'):
    os.mkdir('output')

```sql
SELECT * FROM table
```

In [10]:
output = filter(lambda x: x.startswith('sql'), answer.split('```'))
# Open the file with write permissions
with open('output/player_scd_generation.sql', 'w') as file:
    # Write some data to the file
    file.write('\n'.join(output))